In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor
from pyod.models.copod import COPOD
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import VotingClassifier
import joblib
from scipy import stats
import seaborn as sns
import math
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import warnings

In [ ]:
# --------- DATA READING ---------------------
df = pd.read_csv('data_path')


In [5]:
### GENERAL FUNCTIONS ###

#--------------- Function groups IRG data --------------------------
def filtraRGIs(df):

    DADOS_AGP = df.groupby(['cod_rgi','nome_rgi','UF','ano','epiweek'])['atend_ivas'].sum()
    dataAGP = pd.DataFrame(DADOS_AGP)
    dataAGP.reset_index(inplace = True)

    return dataAGP



##------------------ R(t) ----------------------------------
def calculo_rt(n, m, gamma, q, serie):
    rt = [0] * n
    w = [0.0] * (m + 1)

    for i in range(1, n + 1):
        deno = 0.0
        soma = 0.0
        ie = min(i, m)

        if ie > 1:
            ww = ((1 - q) ** 2) / (q * (1.0 + (ie - 1) * q ** ie - ie * q ** (ie - 1)))
            for j in range(1, ie + 1):
                w[j] = ww * (j - 1) * q ** (j - 1)
                deno += w[j] * serie[i - j]
                soma += w[j]

            if deno != 0:
                rt[i - 1] = soma * serie[i - 1] / deno

    return rt




#--------------- ML -------------------------------------
def treina_Models_ensemble(data_T, data_V):

    ISF = IsolationForest(n_estimators = 500, contamination=0.4)
    LOF = LocalOutlierFactor(n_neighbors = 500, contamination= 0.4)
    OCSVM = OneClassSVM(nu = 0.8, kernel='rbf', gamma=0.001)
    COPO = COPOD(contamination = 0.4)

    ISF.fit(data_T)
    LOF.fit(data_T)
    OCSVM.fit(data_T)
    COPO.fit(data_T)


    ISF_Pred = ISF.predict(data_V)
    LOF_Pred = LOF.fit_predict(data_V)
    OCSVM_Pred = OCSVM.predict(data_V)
    COPO_Pred = COPO.predict(data_V)

    for i in range(len(COPO_Pred)):

        if (COPO_Pred[i] == 1):
            COPO_Pred[i] = -1
        elif (COPO_Pred[i] == 0):
            COPO_Pred[i] = 1


    ENSEMBLE_Pred = ISF_Pred+LOF_Pred+OCSVM_Pred+COPO_Pred


    return ISF_Pred, LOF_Pred, OCSVM_Pred, COPO_Pred, ENSEMBLE_Pred




#------------------ Historical limit (2017 - 2019) ----------------------------------
def calculate_limit_sazonal(df_rgi, alpha=0.05):
    limits = []
    ano = df_rgi['ano']
    epiweek = df_rgi['epiweek']


    for a, w in zip(ano, epiweek):

        if a >= 2020:

            filtered_df = df_rgi[(df_rgi['ano'].isin([2017, 2018, 2019])) & (df_rgi['epiweek'] == w)]
            data_week = filtered_df['atend_ivas']
            media = np.mean(data_week)
            desvio_padrao = np.std(data_week, ddof=0)
            z_alpha = stats.norm.ppf(1 - alpha / 2)
            limit = media + z_alpha * (desvio_padrao / np.sqrt(len(data_week)))
            limits.append(limit)

    return limits




#--------------------- Recent past limit -------------------------------
def calculate_limit(serie, l=5, alpha=0.05):
    n = len(serie)
    z_alpha = stats.norm.ppf(1 - alpha / 2)

    actual_l = min(l, n)
    media_movel = np.convolve(serie, np.ones(actual_l) / actual_l, mode='valid')
    media_movel = np.concatenate(([media_movel[0]] * (actual_l - 1), media_movel))

    variabilidade = [np.std(serie[max(0, i-(actual_l-1)):i+1]) for i in range(n)]
    variabilidade = np.array(variabilidade)

    threshold = media_movel + z_alpha * variabilidade / np.sqrt(actual_l)
    return threshold



## IRG Processing


In [6]:
rgis_capitais = {
    120001,  # Rio Branco, Acre
    270001,  # Maceió, Alagoas
    160001,  # Macapá, Amapá
    130001,  # Manaus, Amazonas
    290001,  # Salvador, Bahia
    230001,  # Fortaleza, Ceará
    530001,  # Brasília, Distrito Federal
    320001,  # Vitória, Espírito Santo
    520001,  # Goiânia, Goiás
    210001,  # São Luís, Maranhão
    510001,  # Cuiabá, Mato Grosso
    500001,  # Campo Grande, Mato Grosso do Sul
    310001,  # Belo Horizonte, Minas Gerais
    150001,  # Belém, Pará
    250001,  # João Pessoa, Paraíba
    410001,  # Curitiba, Paraná
    260001,  # Recife, Pernambuco
    220001,  # Teresina, Piauí
    330001,  # Rio de Janeiro, Rio de Janeiro
    240001,  # Natal, Rio Grande do Norte
    430001,  # Porto Alegre, Rio Grande do Sul
    110001,  # Porto Velho, Rondônia
    140001,  # Boa Vista, Roraima
    420001,  # Florianópolis, Santa Catarina
    350001,  # São Paulo, São Paulo
    280001,  # Aracaju, Sergipe
    170001   # Palmas, Tocantins
}

In [8]:
### MAIN CODE ####
warnings.filterwarnings("ignore")

dadosAGP = filtraRGIs(df)


regioes_unicas = dadosAGP['cod_rgi'].unique()

# DataFrame final
result_final_rgi = pd.DataFrame()

# Processing for each IRG
for regiao_desejada in regioes_unicas:
    df_rgi = dadosAGP[dadosAGP['cod_rgi'] == regiao_desejada]

    # split data
    DADOS_TREINO = df_rgi[(df_rgi['ano'] >= 2017) & (df_rgi['ano'] <= 2019)]
    DADOS_DETECCAO = df_rgi[df_rgi['ano'] >= 2020]

    # Calculations for RT approach from 2020
    n = len(DADOS_DETECCAO)
    m=5
    gamma=0.2
    q = math.exp(-gamma)
    serie = DADOS_DETECCAO['atend_ivas']
    rt = calculo_rt(n, m, gamma, q, serie.to_numpy())

    #-------Limits-------------
    limite1 = calculate_limit_sazonal(df_rgi)
    limite2 = calculate_limit(serie)

    DADOS_DETECCAO['limit1'] = limite1
    DADOS_DETECCAO['limit2'] = limite2
    DADOS_DETECCAO['limite_max'] = DADOS_DETECCAO[['limit1', 'limit2']].max(axis=1)


    scaler1 = MinMaxScaler()
    dados_T = np.reshape(DADOS_TREINO['atend_ivas'].values, (-1, 1))
    dados_T = scaler1.fit_transform(dados_T)

    scaler2 = MinMaxScaler()
    dados_V = np.reshape(DADOS_DETECCAO['atend_ivas'].values, (-1, 1))
    dados_V = scaler2.fit_transform(dados_V)


    ISF_Pred, LOF_Pred, OCSVM_Pred, COPOD_Pred, ENSEMBLE_Pred = treina_Models_ensemble(dados_T, dados_V)
    dados2 = scaler2.inverse_transform(dados_V)

    # -----------Ensemble Final--------------------------------
    VOT1 = -1
    limiar_rt = 1.2

    final_alerta_ML = []
    for i in range(n):
        votos = sum([ ISF_Pred[i] == VOT1, LOF_Pred[i] == VOT1, OCSVM_Pred[i] == VOT1, COPOD_Pred[i] == VOT1])

        is_anomalia = "Sim" if votos >= 2 else "Nao"
        final_alerta_ML.append("Sim" if (is_anomalia == "Sim" and dados2[i][0] > DADOS_DETECCAO['limite_max'].iloc[i]) else "Nao")


    final_alerta = []
    for i in range(n):
        votos = sum([ ISF_Pred[i] == VOT1, LOF_Pred[i] == VOT1, OCSVM_Pred[i] == VOT1, COPOD_Pred[i] == VOT1])
        if rt[i] > limiar_rt:
            votos += 1
        is_anomalia = "Sim" if votos >= 3 else "Nao" #
        final_alerta.append("Sim" if (is_anomalia == "Sim" and dados2[i][0] > DADOS_DETECCAO['limite_max'].iloc[i]) else "Nao")

    #-------------- early warning signals (EWS)--------------------
    DADOS_DETECCAO['EWS_ISF'] = ['Sim' if (ISF_Pred[i] == VOT1 and dados2[i][0] > DADOS_DETECCAO['limite_max'].iloc[i]) else 'Nao' for i in range(n)]
    DADOS_DETECCAO['EWS_LOF'] = ['Sim' if (LOF_Pred[i] == VOT1 and dados2[i][0] > DADOS_DETECCAO['limite_max'].iloc[i]) else 'Nao' for i in range(n)]
    DADOS_DETECCAO['EWS_OCSVM'] = ['Sim' if (OCSVM_Pred[i] == VOT1 and dados2[i][0] > DADOS_DETECCAO['limite_max'].iloc[i]) else 'Nao' for i in range(n)]
    DADOS_DETECCAO['EWS_COPOD'] = ['Sim' if (COPOD_Pred[i] == VOT1 and dados2[i][0] > DADOS_DETECCAO['limite_max'].iloc[i]) else 'Nao' for i in range(n)]
    DADOS_DETECCAO['R(t)'] = rt
    DADOS_DETECCAO['EWS_R(t)'] = ['Sim' if (rt[i] > limiar_rt and dados2[i][0] > DADOS_DETECCAO['limite_max'].iloc[i]) else 'Nao' for i in range(len(rt))]
    DADOS_DETECCAO['EWS_ML'] = final_alerta_ML
    DADOS_DETECCAO['EWS_MMAING'] = final_alerta

    # Salva arquivo de resultado individualizado
    #DADOS_DETECCAO.to_excel(f'ensemble_rgi_{regiao_desejada}.xlsx', index=False)

    # DataFrame final
    result_final_rgi = pd.concat([result_final_rgi, DADOS_DETECCAO])


result_final_rgi.rename(columns={'cod_rgi': 'Code', 'atend_ivas':'PHC encounters','ano': 'year', 'limite_max':'Upperbound','nome_rgi':'Name_irg'}, inplace=True)

# Save final file
result_final_rgi.to_csv('ews.csv', index=False)

